In [3]:
import numpy as np
import pandas as pd
import os
import random
import cv2
import math
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from skimage.transform import rotate

Using TensorFlow backend.


## Get files from a path

In [4]:
def get_files(path):
    files = list()
    
    # /kaggle/input/dataset/train
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    
    return files

In [5]:
train_files_path = get_files('/kaggle/input/dance-form-vaibhav/dataset/train/')

train_csv = pd.read_csv('/kaggle/input/dance-form-vaibhav/dataset/train.csv')

print("Train files : {0}".format(len(train_files_path)))

Train files : 364


In [6]:
train_dict = dict(zip(train_csv.Image, train_csv.target))
print(train_dict['96.jpg'])

manipuri


### Split into training and validation set

In [7]:
split = int(len(train_files_path) * 0.83)
random.shuffle(train_files_path)
train_files = train_files_path[:split]
val_files = train_files_path[split:]

print("Training files {0}".format(len(train_files)))
print("Validation files {0}".format(len(val_files)))

Training files 302
Validation files 62


### Converting dance forms into numbers

In [8]:
unique_labels = list(set(train_dict.values()))
unique_labels_mapping = dict()
for index in range(len(unique_labels)):
    unique_labels_mapping[unique_labels[index]] = index

total_classes = len(unique_labels)
print("Number of classes : {0}".format(total_classes))

Number of classes : 8


In [9]:
def get_one_hot_encoded_mask(value, num_labels):
    return to_categorical(value, num_classes = num_labels)

### Setting up hyper-parameters

In [10]:
img_width, img_height = 256, 256
batch_size = 16
epochs = 10
learning_rate = 1e-3

### Creating Augmentations

In [11]:
def rotate_image(image, angle_list):
    rotated_images = list()
    for angle in angle_list:
        rotated_images.append(rotate(image,angle))
    
    return rotated_images

In [12]:
def scaleDown_image(image, fx=0.6, fy=0.6):
    return cv2.resize(image, None, fx= 0.6, fy= 0.6, interpolation= cv2.INTER_LINEAR)

In [13]:
def scaleUp_image(image, fx = 2, fy = 2):
    return cv2.resize(image, None, fx = fx, fy = fy, interpolation= cv2.INTER_LINEAR)

In [14]:
def horizontal_flip(img):
    return img[: , ::-1]

## Creating DataGenerator

In [15]:
class DataGenerator:
    def __init__(self, train_files, valid_files, labels_dict, batch_size = 16, val_augment = True):
        self.train_files = train_files
        self.valid_files = valid_files
        self.labels_dict = labels_dict
        self.batch_size = batch_size
        self.val_augment = val_augment
        
    def train_generator(self):
        num_images = len(self.train_files)
        while True:
            x_batch = list()
            y_batch = list()
            index_list = list(range(0,num_images))
            index_list = shuffle(index_list)
            for idxs in range(0, num_images, self.batch_size):
                x_batch = list()
                y_batch = list()
                for idx in index_list[idxs:min(idxs+self.batch_size, num_images)]:
                    
                    img = cv2.imread(self.train_files[idx])
                    img = cv2.resize(img, (img_width, img_height))
                    x_batch.append(img)
                    
                    image_name = self.train_files[idx].split("/")
                    label = unique_labels_mapping[train_dict[str(image_name[-1])]]
                    label = get_one_hot_encoded_mask(label, total_classes)
                    y_batch.append(label)
                    
                    rotated_images = rotate_image(img, [45, 60, -45, -60])
                    for rotated_image in rotated_images:
                        x_batch.append(rotated_image)
                        y_batch.append(label)
                                        
                    x_batch.append(horizontal_flip(img))
                    y_batch.append(label)
                
                x_batch, y_batch = shuffle(x_batch, y_batch, random_state=0)
                yield (np.asarray(x_batch), np.asarray(y_batch))
    
    def valid_generator(self):
        num_images = len(self.valid_files)
        while True:
            x_batch = list()
            y_batch = list()
            index_list = list(range(0,num_images))
            index_list = shuffle(index_list)
            for idxs in range(0, num_images, self.batch_size):
                x_batch = list()
                y_batch = list()
                for idx in index_list[idxs:min(idxs+self.batch_size, num_images)]:
                    
                    img = cv2.imread(self.valid_files[idx])
                    img = cv2.resize(img, (img_width, img_height))
                    x_batch.append(img)
                    
                    image_name = self.valid_files[idx].split("/")
                    label = unique_labels_mapping[train_dict[str(image_name[-1])]]
                    label = get_one_hot_encoded_mask(label, total_classes)
                    y_batch.append(label)
                    
                    if self.val_augment:
                        rotated_images = rotate_image(img, [45, 60, -45, -60])
                        for rotated_image in rotated_images:
                            x_batch.append(rotated_image)
                            y_batch.append(label)

                        x_batch.append(horizontal_flip(img))
                        y_batch.append(label)
                        
                x_batch, y_batch = shuffle(x_batch, y_batch, random_state=0)
                yield (np.asarray(x_batch), np.asarray(y_batch))

In [16]:
epoch_steps = int(math.ceil(len(train_files) / batch_size) * 6)
print(epoch_steps)
val_steps = int(math.ceil(len(val_files) / batch_size) * 6)
print(val_steps)

114
24


## Model Experiments

In [17]:
from keras.applications import vgg19, resnet50, inception_v3, inception_resnet_v2, mobilenet_v2
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras.optimizers import Adam, SGD
from keras.models import Model
import tensorflow.keras as keras

### VGG19 Setup

In [ ]:
model = vgg19.VGG19(include_top=False, input_shape=(img_width, img_height, 3), \
                    weights='/kaggle/input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')
model.summary()

In [ ]:
x = model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
x = Dense(256,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

In [ ]:
model=Model(inputs=model.input,outputs=final_layer)
model.summary()

In [ ]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [ ]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=10, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2)

### ResNet50 Setup

In [ ]:
base_model = resnet50.ResNet50(include_top=False, input_shape=(img_width, img_height, 3),
                    weights='/kaggle/input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=final_layer)
model.summary()

In [ ]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [ ]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=20, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2)

### InceptionV3 Setup

In [18]:
base_model = inception_v3.InceptionV3(include_top=False, input_shape=(img_width, img_height, 3),
                    weights='/kaggle/input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation='relu', kernel_regularizer='l1')(x)
x = Dropout(0.5)(x)
x = Dense(256,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=final_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [19]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [21]:
def lr_scheduler(epoch, lr):
#     if epoch % 2 == 0:
#         return lr
    return lr * 1.0

In [22]:
callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
#     keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/models.{epoch:03d}.hdf5',
#                     monitor='loss', verbose=2, save_best_only=False)
]

In [23]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=20, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2, callbacks=callbacks)

Epoch 1/20
114/114 - 113s - loss: 591.3223 - accuracy: 0.1947 - val_loss: 13.3516 - val_accuracy: 0.2285 - lr: 0.0010
Epoch 2/20
114/114 - 111s - loss: 8.7055 - accuracy: 0.2405 - val_loss: 8.6045 - val_accuracy: 0.2478 - lr: 0.0010
Epoch 3/20
114/114 - 110s - loss: 8.1746 - accuracy: 0.3366 - val_loss: 8.2892 - val_accuracy: 0.3006 - lr: 0.0010
Epoch 4/20
114/114 - 110s - loss: 7.7963 - accuracy: 0.4390 - val_loss: 8.0916 - val_accuracy: 0.3141 - lr: 0.0010
Epoch 5/20
114/114 - 109s - loss: 7.5372 - accuracy: 0.5322 - val_loss: 10.3456 - val_accuracy: 0.2554 - lr: 0.0010
Epoch 6/20
114/114 - 110s - loss: 7.1572 - accuracy: 0.6527 - val_loss: 8.0314 - val_accuracy: 0.4068 - lr: 0.0010
Epoch 7/20
114/114 - 109s - loss: 6.7462 - accuracy: 0.7457 - val_loss: 10.1997 - val_accuracy: 0.3073 - lr: 0.0010
Epoch 8/20
114/114 - 108s - loss: 6.4010 - accuracy: 0.8258 - val_loss: 10.8890 - val_accuracy: 0.3329 - lr: 0.0010
Epoch 9/20
114/114 - 110s - loss: 5.9887 - accuracy: 0.8997 - val_loss: 7.

KeyboardInterrupt: 

### InceptionResnetV2 Setup

In [33]:
base_model = inception_resnet_v2.InceptionResNetV2(include_top=False, input_shape=(img_width, img_height, 3),
                    weights='/kaggle/input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=final_layer)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 127, 127, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 127, 127, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 127, 127, 32) 0           batch_normalization_203[0][0]    
____________________________________________________________________________________________

                                                                 block8_8_conv[0][0]              
__________________________________________________________________________________________________
block8_8_ac (Activation)        (None, 6, 6, 2080)   0           block8_8[0][0]                   
__________________________________________________________________________________________________
conv2d_399 (Conv2D)             (None, 6, 6, 192)    399360      block8_8_ac[0][0]                
__________________________________________________________________________________________________
batch_normalization_399 (BatchN (None, 6, 6, 192)    576         conv2d_399[0][0]                 
__________________________________________________________________________________________________
activation_399 (Activation)     (None, 6, 6, 192)    0           batch_normalization_399[0][0]    
__________________________________________________________________________________________________
conv2d_400

In [34]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [35]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [36]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=10, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2)

Epoch 1/10
114/114 - 146s - loss: 1.8158 - accuracy: 0.2899 - val_loss: 2.2578 - val_accuracy: 0.1801
Epoch 2/10
114/114 - 143s - loss: 1.2469 - accuracy: 0.5292 - val_loss: 1.8476 - val_accuracy: 0.3580
Epoch 3/10
114/114 - 143s - loss: 0.8863 - accuracy: 0.6709 - val_loss: 1.8980 - val_accuracy: 0.3477
Epoch 4/10
114/114 - 144s - loss: 0.5055 - accuracy: 0.8239 - val_loss: 2.0544 - val_accuracy: 0.3656
Epoch 5/10


KeyboardInterrupt: 

### MobileNetV2 Setup

In [39]:
base_model = mobilenet_v2.MobileNetV2(include_top=False, input_shape=(img_width, img_height, 3),
                    weights='/kaggle/input/mobilenet-v2-128/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_128_no_top.h5')
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=final_layer)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 257, 257, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 128, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 128, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [40]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [41]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [42]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=10, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2)

Epoch 1/10
114/114 - 115s - loss: 1.2873 - accuracy: 0.5614 - val_loss: 3.0569 - val_accuracy: 0.1156
Epoch 2/10
114/114 - 115s - loss: 0.1571 - accuracy: 0.9546 - val_loss: 3.2391 - val_accuracy: 0.1393
Epoch 3/10


KeyboardInterrupt: 

## Generate Predictions

In [25]:
inv_map = {v: k for k, v in unique_labels_mapping.items()}
print(inv_map)

{0: 'manipuri', 1: 'bharatanatyam', 2: 'odissi', 3: 'sattriya', 4: 'kathak', 5: 'kuchipudi', 6: 'kathakali', 7: 'mohiniyattam'}


In [26]:
test_csv = pd.read_csv('/kaggle/input/dance-form-vaibhav/dataset/test.csv')

In [29]:
test_images = test_csv['Image']
print(test_images[:5])

0    508.jpg
1    246.jpg
2    473.jpg
3    485.jpg
4    128.jpg
Name: Image, dtype: object


In [34]:
output = list()
for index in range(len(test_images)):
    img = cv2.imread('/kaggle/input/dance-form-vaibhav/dataset/test/' + str(test_images[index]))
    img = cv2.resize(img, (img_width, img_height))
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)
    output.append(inv_map[np.argmax(pred)])

In [37]:
df = pd.DataFrame({'Image': test_images,
                   'target': output})
df.to_csv("/kaggle/working/output.csv", index=False)